# Multiariate Multiple Step MLP Model

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


#### split a multivariate sequence into samples

In [2]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
            
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

## 1. Prepare data

#### Define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
in_seq1

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

#### convert to [rows, columns] structure

In [5]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [6]:
in_seq1

array([[10],
       [20],
       [30],
       [40],
       [50],
       [60],
       [70],
       [80],
       [90]])

#### horizontally stack columns

In [7]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))

#### choose a number of time steps

In [8]:
n_steps_in, n_steps_out = 3, 2

#### convert into input/output

In [9]:
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

In [10]:
for i in range(len(X)):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


#### flatten input

In [11]:
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [12]:
X

array([[10, 15, 20, 25, 30, 35],
       [20, 25, 30, 35, 40, 45],
       [30, 35, 40, 45, 50, 55],
       [40, 45, 50, 55, 60, 65],
       [50, 55, 60, 65, 70, 75],
       [60, 65, 70, 75, 80, 85]])

## 2. Define model

In [13]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

## 3. Fit model

In [14]:
model.fit(X, y, epochs=2000, verbose=0)

## 3. Demonstrate prediction

In [15]:
x_input = np.array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[185.85703 207.08229]]
